In [1]:
# Cell 1: Imports
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import math
import os
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path


# Create directory for datasets
os.makedirs('uploaded_dataset', exist_ok=True)

In [2]:
# Cell 2: Upload widget
uploader = widgets.FileUpload(
    accept='.mp4,.avi,.mov', 
    multiple=False
)
display(uploader)

import ipywidgets as widgets
from IPython.display import display

label_selector = widgets.Dropdown(
    options=[('Normal', 0), ('Abnormal', 1)],
    description='Gait Type:',
    style={'description_width': 'initial'}
)

display(label_selector)


FileUpload(value=(), accept='.mp4,.avi,.mov', description='Upload')

Dropdown(description='Gait Type:', options=(('Normal', 0), ('Abnormal', 1)), style=DescriptionStyle(descriptio…

In [3]:
# Cell 3: Save uploaded video
video_path = None

if uploader.value:
    uploaded_file = uploader.value[0]
    video_name = uploaded_file.name
    video_path = f"uploaded_dataset/{video_name}"
    with open(video_path, 'wb') as f:
        f.write(uploaded_file.content)
    print(f"Saved to: {video_path}")

Saved to: uploaded_dataset/AnteriorWalkReg.mp4


In [4]:
# Cell 4: Feature extraction function
def extract_gait_features(video_path, label=None):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False)
    scale_factor = 0.5

    def resize_image(image, scale_factor):
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        return cv2.resize(image, (new_width, new_height))

    def calculate_angle(a, b, c):
        a, b, c = np.array(a), np.array(b), np.array(c)
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
        return 360 - angle if angle > 180.0 else angle

    def get_coords(landmarks, index):
        return (landmarks[index].x, landmarks[index].y)

    # MediaPipe keypoint indices
    RIGHT_HIP, RIGHT_KNEE, RIGHT_ANKLE = 24, 26, 28
    LEFT_HIP, LEFT_KNEE, LEFT_ANKLE = 23, 25, 27

    cap = cv2.VideoCapture(video_path)
    prev_angle, prev_angular_velocity, prev_time = None, None, 0
    data_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = resize_image(frame, scale_factor)
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            rh, rk, ra = get_coords(lm, RIGHT_HIP), get_coords(lm, RIGHT_KNEE), get_coords(lm, RIGHT_ANKLE)
            lh, lk, la = get_coords(lm, LEFT_HIP), get_coords(lm, LEFT_KNEE), get_coords(lm, LEFT_ANKLE)

            rk_angle = calculate_angle(rh, rk, ra)
            lk_angle = calculate_angle(lh, lk, la)

            v1 = (rk[0] - ra[0], rk[1] - ra[1])
            v2 = (rh[0] - ra[0], rh[1] - ra[1])
            dot = v1[0]*v2[0] + v1[1]*v2[1]
            mag1 = math.sqrt(v1[0]**2 + v1[1]**2)
            mag2 = math.sqrt(v2[0]**2 + v2[1]**2)
            angle_rad = math.acos(dot / (mag1 * mag2 + 1e-6))
            angle_deg = math.degrees(angle_rad)

            time_diff = current_time - prev_time
            angular_velocity = None
            if prev_angle is not None and time_diff > 0:
                angular_velocity = (angle_deg - prev_angle) / time_diff

            linear_acceleration = None
            if prev_angular_velocity is not None and angular_velocity is not None:
                linear_acceleration = (angular_velocity - prev_angular_velocity)

            prev_angle = angle_deg
            prev_angular_velocity = angular_velocity
            prev_time = current_time

            data_list.append({
                'time': current_time,
                'right_knee_angle': rk_angle,
                'left_knee_angle': lk_angle,
                'angular_velocity': angular_velocity,
                'linear_acceleration': linear_acceleration,
                'label': label  # 0 for normal, 1 for abnormal
            })

    cap.release()
    pose.close()

    df = extract_gait_features(video_path, label=label_selector.value)
    csv_name = Path(video_path).stem + '_gait.csv'
    if output_csv:
        df.to_csv(f'uploaded_dataset/{csv_name}', index=False)
        print(f"Saved: uploaded_dataset/{csv_name}")
    return df


In [ ]:
# Cell 5: Run feature extraction
if video_path:
    df = extract_gait_features(video_path)
    display(df.head())


C:\Users\Sandra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
# Cell 6: Prediction
features = df[['right_knee_angle', 'left_knee_angle', 'angular_velocity', 'linear_acceleration']].mean().values.reshape(1, -1)
features_scaled = scaler.transform(features)
prediction = clf.predict(features_scaled)
print("Prediction:", "Normal" if prediction[0] == 0 else "Abnormal")
